In [1]:
import sys
import logging

import numpy as np
import scipy as sp
import sklearn
import statsmodels.api as sm
from statsmodels.formula.api import ols

%load_ext autoreload
%autoreload 2

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import seaborn as sns
sns.set_context("poster")
sns.set(rc={'figure.figsize': (16, 9.)})
sns.set_style("whitegrid")

import pandas as pd
pd.set_option("display.max_rows", 120)
pd.set_option("display.max_columns", 120)

logging.basicConfig(level=logging.INFO, stream=sys.stdout)

In [2]:
from lrann.datasets import Loader, random_train_test_split
from lrann.estimators import ImplicitEst
from lrann.models import BilinearNet, DeepNet
from lrann.evaluation import mrr_score, precision_recall_score
from lrann.utils import is_cuda_available

In [3]:
data = Loader().load_movielens('100k-old')

In [4]:
train, test = random_train_test_split(data)

## Estimator using a low-rank approximation model

In [5]:
lra_model = BilinearNet(data.n_users, data.n_items, embedding_dim=32, sparse=False)
lra_est = ImplicitEst(model=lra_model, 
                      n_iter=5, 
                      use_cuda=is_cuda_available())
lra_est.fit(train, verbose=True)

Epoch 0: loss 0.27675932797193525
Epoch 1: loss 0.17167725524902344
Epoch 2: loss 0.15408831919431687
Epoch 3: loss 0.14492063223719598
Epoch 4: loss 0.13740948897600175


In [6]:
prec, recall = precision_recall_score(lra_est, test)
prec.mean(), recall.mean()

(0.11367974549310712, 0.08312115728535541)

In [7]:
mrr_score(lra_est, train).mean(), mrr_score(lra_est, test).mean()

(0.05072063447153412, 0.03976879794850387)

## Estimator using a deep neural model

In [8]:
nn_model = DeepNet(data.n_users, data.n_items, embedding_dim=32, sparse=False)
nn_est = ImplicitEst(model=nn_model, 
                     n_iter=5, 
                     use_cuda=is_cuda_available())
nn_est.fit(train, verbose=True)

Epoch 0: loss 0.23424265429973604
Epoch 1: loss 0.21347526460886002
Epoch 2: loss 0.2105945165872574
Epoch 3: loss 0.20520013264417647
Epoch 4: loss 0.20541262471675872


In [9]:
prec, recall = precision_recall_score(nn_est, test)
prec.mean(), recall.mean()

(0.08833510074231177, 0.05972723965633153)

In [10]:
mrr_score(lra_est, train).mean(), mrr_score(nn_est, test).mean()

(0.05072063447153412, 0.02856018978872365)